<a href="https://colab.research.google.com/github/stanislavlia/churn_rate_mlops/blob/main/churn_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime


In [2]:
DATA_PATH="/content/drive/MyDrive/churn_data/"




### Process & aggregate user logs




In [3]:
user_logs_df = pd.read_csv(DATA_PATH + "user_logs_v2.csv")

In [4]:
user_logs_df.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,20170331,8,4,0,1,21,18,6309.273
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,20170330,2,2,1,0,9,11,2390.699
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,20170331,52,3,5,3,84,110,23203.337
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,20170331,176,4,2,2,19,191,7100.454
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,20170331,2,1,0,1,112,93,28401.558


In [9]:
#convert string to datetime

user_logs_df["date"] = user_logs_df["date"].apply(lambda x : datetime.strptime(str(x), "%Y%m%d"))

In [11]:
user_logs_df["day_of_week"] = user_logs_df["date"].apply(lambda x: x.weekday())
user_logs_df["day_of_week_name"] = user_logs_df["date"].apply(lambda x: x.strftime("%A"))
user_logs_df["month"] = user_logs_df["date"].apply(lambda x: x.month)
user_logs_df["month_name"] = user_logs_df["date"].apply(lambda x: x.strftime("%B"))
user_logs_df["is_weekend"] = user_logs_df["date"].apply(lambda x: x.weekday() >= 5)
user_logs_df["day_of_month"] = user_logs_df["date"].apply(lambda x: x.day)
user_logs_df["quarter"] = user_logs_df["date"].apply(lambda x: (x.month - 1) // 3 + 1)
user_logs_df["day_of_year"] = user_logs_df["date"].apply(lambda x: x.timetuple().tm_yday)


In [18]:
from scipy import stats

def mode(series):
    m = stats.mode(series, nan_policy='omit')
    if isinstance(m.mode, np.ndarray) and len(m.mode) > 0:
        return m.mode[0]
    elif not isinstance(m.mode, np.ndarray):
        return m.mode
    else:
        return None

In [13]:
user_logs_df.head(3)

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,day_of_week,day_of_week_name,month,month_name,is_weekend,day_of_month,quarter,day_of_year
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,2017-03-31,8,4,0,1,21,18,6309.273,4,Friday,3,March,False,31,1,90
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,2017-03-30,2,2,1,0,9,11,2390.699,3,Thursday,3,March,False,30,1,89
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,2017-03-31,52,3,5,3,84,110,23203.337,4,Friday,3,March,False,31,1,90


In [15]:
user_logs_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18396362 entries, 0 to 18396361
Data columns (total 17 columns):
 #   Column            Non-Null Count     Dtype         
---  ------            --------------     -----         
 0   msno              18396362 non-null  object        
 1   date              18396362 non-null  datetime64[ns]
 2   num_25            18396362 non-null  int64         
 3   num_50            18396362 non-null  int64         
 4   num_75            18396362 non-null  int64         
 5   num_985           18396362 non-null  int64         
 6   num_100           18396362 non-null  int64         
 7   num_unq           18396362 non-null  int64         
 8   total_secs        18396362 non-null  float64       
 9   day_of_week       18396362 non-null  int64         
 10  day_of_week_name  18396362 non-null  object        
 11  month             18396362 non-null  int64         
 12  month_name        18396362 non-null  object        
 13  is_weekend        1839636

In [19]:
aggregated_df = user_logs_df.groupby('msno').agg({
    'num_25': ['mean'],
    'num_50': ['mean', 'min', 'max', 'std'],
    'num_75': ['mean'],
    'num_985': ['mean'],
    'num_100': ['mean'],
    'num_unq': ['mean',],
    'total_secs': ['mean', 'std'],
    'month' : [mode],
    'is_weekend' : ["mean", "count", "sum"],
    "quarter" : [mode],
    "day_of_week" : [mode],
    "day_of_year" : [mode, "min", "max", "count"]
}).reset_index()

In [20]:
aggregated_df.head(3)

msno    num_25    num_50          \
                                                     mean      mean min max   
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=  3.307692  0.423077   0   2   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=  6.161290  2.903226   0  12   
2  +++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=  1.535714  0.428571   0   3   

               num_75   num_985    num_100    num_unq  ... month is_weekend  \
        std      mean      mean       mean       mean  ...  mode       mean   
0  0.643309  0.384615  0.192308  18.153846  20.384615  ...     3   0.307692   
1  2.494726  2.419355  4.645161  19.000000  28.548387  ...     3   0.258065   
2  0.835711  0.535714  0.428571  17.321429  16.714286  ...     3   0.285714   

            quarter day_of_week day_of_year                
  count sum    mode        mode        mode min max count  
0    26   8       1           2          60  60  90    26  
1    31   8       1           2          60  60  90    31  
2    28   8       1           3          60  60  90    28  

[3 rows x 22 columns]

In [21]:
aggregated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103894 entries, 0 to 1103893
Data columns (total 22 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   (msno, )              1103894 non-null  object 
 1   (num_25, mean)        1103894 non-null  float64
 2   (num_50, mean)        1103894 non-null  float64
 3   (num_50, min)         1103894 non-null  int64  
 4   (num_50, max)         1103894 non-null  int64  
 5   (num_50, std)         1016672 non-null  float64
 6   (num_75, mean)        1103894 non-null  float64
 7   (num_985, mean)       1103894 non-null  float64
 8   (num_100, mean)       1103894 non-null  float64
 9   (num_unq, mean)       1103894 non-null  float64
 10  (total_secs, mean)    1103894 non-null  float64
 11  (total_secs, std)     1016672 non-null  float64
 12  (month, mode)         1103894 non-null  int64  
 13  (is_weekend, mean)    1103894 non-null  float64
 14  (is_weekend, count)   1103894 non-

In [29]:
aggregated_df.columns = [
    'msno',
    'logs_num_25_mean',
    'logs_num_50_mean',
    'logs_num_50_min',
    'logs_num_50_max',
    'logs_num_50_std',
    'logs_num_75_mean',
    'logs_num_985_mean',
    'logs_num_100_mean',
    'logs_num_unq_mean',
    'logs_total_secs_mean',
    'logs_total_secs_std',
    'logs_month_mode',
    'logs_is_weekend_mean',
    'logs_is_weekend_count',
    'logs_is_weekend_sum',
    'logs_quarter_mode',
    'logs_day_of_week_mode',
    'logs_day_of_year_mode',
    'logs_day_of_year_min',
    'logs_day_of_year_max',
    'logs_day_of_year_count'
]


In [30]:
aggregated_df.head()

,msno,logs_num_25_mean,logs_num_50_mean,logs_num_50_min,logs_num_50_max,logs_num_50_std,logs_num_75_mean,logs_num_985_mean,logs_num_100_mean,logs_num_unq_mean,...,logs_month_mode,logs_is_weekend_mean,logs_is_weekend_count,logs_is_weekend_sum,logs_quarter_mode,logs_day_of_week_mode,logs_day_of_year_mode,logs_day_of_year_min,logs_day_of_year_max,logs_day_of_year_count
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,3.307692,0.423077,0,2,0.643309,0.384615,0.192308,18.153846,20.384615,...,3,0.307692,26,8,1,2,60,60,90,26
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,6.161290,2.903226,0,12,2.494726,2.419355,4.645161,19.000000,28.548387,...,3,0.258065,31,8,1,2,60,60,90,31
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,1.535714,0.428571,0,3,0.835711,0.535714,0.428571,17.321429,16.714286,...,3,0.285714,28,8,1,3,60,60,90,28
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,9.857143,7.761905,0,60,13.827164,4.761905,3.047619,20.761905,39.428571,...,3,0.333333,21,7,1,6,61,61,89,21
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,3.620690,0.827586,0,5,1.226754,1.344828,1.206897,16.517241,7.931034,...,3,0.241379,29,7,1,2,60,60,90,29


In [31]:
aggregated_df.to_csv(DATA_PATH + "aggregated_logs.csv")

### Process & aggregate user transactions

In [32]:
user_transactions_df = pd.read_csv(DATA_PATH + "transactions_v2.csv")

In [33]:
user_transactions_df.head(3)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0


In [34]:
#conver to Datetime obj

user_transactions_df["transaction_date"] = user_transactions_df["transaction_date"].apply(lambda x : datetime.strptime(str(x), "%Y%m%d"))
user_transactions_df["membership_expire_date"] = user_transactions_df["membership_expire_date"].apply(lambda x : datetime.strptime(str(x), "%Y%m%d"))


In [35]:
# Features for 'transaction_date'
user_transactions_df["transaction_day_of_week"] = user_transactions_df["transaction_date"].apply(lambda x: x.weekday())
user_transactions_df["transaction_day_of_week_name"] = user_transactions_df["transaction_date"].apply(lambda x: x.strftime("%A"))
user_transactions_df["transaction_month"] = user_transactions_df["transaction_date"].apply(lambda x: x.month)
user_transactions_df["transaction_month_name"] = user_transactions_df["transaction_date"].apply(lambda x: x.strftime("%B"))
user_transactions_df["transaction_is_weekend"] = user_transactions_df["transaction_date"].apply(lambda x: x.weekday() >= 5)
user_transactions_df["transaction_day_of_month"] = user_transactions_df["transaction_date"].apply(lambda x: x.day)
user_transactions_df["transaction_quarter"] = user_transactions_df["transaction_date"].apply(lambda x: (x.month - 1) // 3 + 1)
user_transactions_df["transaction_day_of_year"] = user_transactions_df["transaction_date"].apply(lambda x: x.timetuple().tm_yday)

# Features for 'membership_expire_date'
user_transactions_df["membership_expire_day_of_week"] = user_transactions_df["membership_expire_date"].apply(lambda x: x.weekday())
user_transactions_df["membership_expire_day_of_week_name"] = user_transactions_df["membership_expire_date"].apply(lambda x: x.strftime("%A"))
user_transactions_df["membership_expire_month"] = user_transactions_df["membership_expire_date"].apply(lambda x: x.month)
user_transactions_df["membership_expire_month_name"] = user_transactions_df["membership_expire_date"].apply(lambda x: x.strftime("%B"))
user_transactions_df["membership_expire_is_weekend"] = user_transactions_df["membership_expire_date"].apply(lambda x: x.weekday() >= 5)
user_transactions_df["membership_expire_day_of_month"] = user_transactions_df["membership_expire_date"].apply(lambda x: x.day)
user_transactions_df["membership_expire_quarter"] = user_transactions_df["membership_expire_date"].apply(lambda x: (x.month - 1) // 3 + 1)
user_transactions_df["membership_expire_day_of_year"] = user_transactions_df["membership_expire_date"].apply(lambda x: x.timetuple().tm_yday)



In [37]:
user_transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1431009 entries, 0 to 1431008
Data columns (total 25 columns):
 #   Column                              Non-Null Count    Dtype         
---  ------                              --------------    -----         
 0   msno                                1431009 non-null  object        
 1   payment_method_id                   1431009 non-null  int64         
 2   payment_plan_days                   1431009 non-null  int64         
 3   plan_list_price                     1431009 non-null  int64         
 4   actual_amount_paid                  1431009 non-null  int64         
 5   is_auto_renew                       1431009 non-null  int64         
 6   transaction_date                    1431009 non-null  datetime64[ns]
 7   membership_expire_date              1431009 non-null  datetime64[ns]
 8   is_cancel                           1431009 non-null  int64         
 9   transaction_day_of_week             1431009 non-null  int64         

In [49]:


aggregated_trans_df = user_transactions_df.groupby('msno').agg({
    'payment_method_id': ['mean'],
    'plan_list_price': ['mean', 'std'],
    'actual_amount_paid': ['mean', 'std'],
    'is_auto_renew': ['mean', 'sum'],
    'is_cancel': ['mean', 'sum'],
    'transaction_day_of_week': ["mean"],
    'transaction_month': ["mean"],
    'transaction_is_weekend': ['mean', 'sum'],
    'transaction_day_of_month': ['mean', 'std'],
    'transaction_day_of_year': ["min", "max"],
    'membership_expire_day_of_week': ["mean"],
    'membership_expire_month': ["mean", "count"],
}).reset_index()

In [51]:
aggregated_trans_df.columns = [
    'msno',
    'trans_payment_method_id_mean',
    'trans_plan_list_price_mean', 'trans_plan_list_price_std',
    'trans_actual_amount_paid_mean', 'trans_actual_amount_paid_std',
    'trans_is_auto_renew_mean', 'trans_is_auto_renew_sum',
    'trans_is_cancel_mean', 'trans_is_cancel_sum',
    'trans_transaction_day_of_week_mean',
    'trans_transaction_month_mean',
    'trans_transaction_is_weekend_mean', 'trans_transaction_is_weekend_sum',
    'trans_transaction_day_of_month_mean', 'trans_transaction_day_of_month_std',
    'trans_transaction_day_of_year_min', 'trans_transaction_day_of_year_max',
    'trans_membership_expire_day_of_week_mean',
    'trans_membership_expire_month_mean', 'trans_membership_expire_month_count'
]


In [52]:
aggregated_trans_df.head()

,msno,trans_payment_method_id_mean,trans_plan_list_price_mean,trans_plan_list_price_std,trans_actual_amount_paid_mean,trans_actual_amount_paid_std,trans_is_auto_renew_mean,trans_is_auto_renew_sum,trans_is_cancel_mean,trans_is_cancel_sum,...,trans_transaction_month_mean,trans_transaction_is_weekend_mean,trans_transaction_is_weekend_sum,trans_transaction_day_of_month_mean,trans_transaction_day_of_month_std,trans_transaction_day_of_year_min,trans_transaction_day_of_year_max,trans_membership_expire_day_of_week_mean,trans_membership_expire_month_mean,trans_membership_expire_month_count
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22.0,1599.0,NaN,1599.0,NaN,0.0,0,0.0,0,...,10.0,1.0,1,23.0,NaN,297,297,1.0,2.0,1
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41.0,99.0,NaN,99.0,NaN,1.0,1,0.0,0,...,3.0,0.0,0,15.0,NaN,74,74,5.0,4.0,1
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39.0,149.0,0.0,149.0,0.0,1.0,2,0.0,0,...,2.5,0.0,0,29.5,2.12132,59,90,3.0,4.5,2
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41.0,149.0,NaN,149.0,NaN,1.0,1,0.0,0,...,3.0,1.0,1,26.0,NaN,85,85,2.0,4.0,1
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41.0,149.0,NaN,149.0,NaN,1.0,1,0.0,0,...,3.0,0.0,0,15.0,NaN,74,74,5.0,4.0,1


In [54]:
aggregated_trans_df.to_csv(DATA_PATH + "aggregated_trans.csv", index=False)


### Merge all data together


In [55]:
members_df = pd.read_csv(DATA_PATH + "members_v3.csv")

In [56]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [57]:
members_df["registration_init_time"] = members_df["registration_init_time"].apply(lambda x : datetime.strptime(str(x), "%Y%m%d"))


In [58]:
members_df["registration_year"] = members_df["registration_init_time"].apply(lambda x: x.year)
members_df["registration_month"] = members_df["registration_init_time"].apply(lambda x: x.month)


In [59]:

full_data_df = pd.merge(members_df, aggregated_trans_df, how="left", on="msno")
full_data_df = pd.merge(full_data_df, aggregated_df, on="msno", how="left")

In [60]:
full_data_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,registration_year,registration_month,trans_payment_method_id_mean,trans_plan_list_price_mean,...,logs_month_mode,logs_is_weekend_mean,logs_is_weekend_count,logs_is_weekend_sum,logs_quarter_mode,logs_day_of_week_mode,logs_day_of_year_mode,logs_day_of_year_min,logs_day_of_year_max,logs_day_of_year_count
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,2011-09-11,2011,9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,2011-09-14,2011,9,41.0,129.0,...,3.0,0.266667,30.0,8.0,1.0,2.0,60.0,60.0,90.0,30.0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,2011-09-15,2011,9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,2011-09-15,2011,9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,2011-09-15,2011,9,40.0,149.0,...,3.0,0.333333,9.0,3.0,1.0,1.0,73.0,73.0,87.0,9.0


In [ ]:
full_data_df.to_csv(DATA_PATH + 'all_data_merged.csv', float_format='%.3f', index=False)